<a href="https://colab.research.google.com/github/jakiyaa/m3u8-downloader/blob/main/m3u8_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://github.com/superyngo/m3u8d">
<img src="https://github.com/fluidicon.png" height=100px, align="left"></a>



<center><h1><b>M3u8 online Downloader </h1></b></center>

In [ ]:
#@title ← Run <Center>Mount Google Drive </Center>
#@markdown <Center> <img src="https://kstatic.googleusercontent.com/files/e20dbc1db7b6eab22fb2d07df8043f8a74b301dbb8c176e6a1992b2dc229c5ae94b5c260cf549c41712fa40b7e639166c6a8461303a27c2a3b9c55b8ca155ec3" class="icon" alt="Icon">

from google.colab import drive
drive.mount('/content/drive')

import os
os.makedirs('drive/MyDrive/m3u8d', exist_ok=True)

In [ ]:
#@title ← Run <Center>Download m3u8-downloader Tool</Center>

!wget -q -O /content/m3u8 https://github.com/llychao/m3u8-downloader/releases/latest/download/m3u8-linux-amd64
!chmod +x /content/m3u8
!pip install -q yt-dlp "yt-dlp[default,curl-cffi]"
print("m3u8-downloader and yt-dlp ready!")

In [ ]:
#@title ← Run <Center>Get M3U8 Links from URL</Center>
import requests
import re

page_url = "" #@param {type:"string"}

try:
    response = requests.get(page_url, timeout=10)
    m3u8_links = re.findall(r'https?://[^"]+\.m3u8', response.text)
    if m3u8_links:
        print(f"Found {len(m3u8_links)} m3u8 link(s):")
        for i, link in enumerate(m3u8_links, 1):
            print(f"{i}. {link}")
    else:
        print("No m3u8 links found.")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
#@title ← Run <Center>Path Settings</Center>
#@markdown 使用相對路徑（從 /content/ 開始），避免路徑重複問題

output_dir = "drive/MyDrive/m3u8d" #@param {type:"string"}

In [ ]:
#@title ← Run <center> <font color='red'> Single File Download </font> <img src='https://play-lh.googleusercontent.com/vi_F_8raK3dL9D32R1VJRmWv18VKvc3niKDOE5z6yQsHOyzH0FQ1vq91eeakoEIFuxH9' height="40" /> </center> { display-mode: "form" }
import os
from datetime import datetime

M3u8_link = "" #@param {type:"string"}
filename = "video.mp4" #@param {type:"string"}

output_path = os.path.join(output_dir, filename)

result = !"/content/m3u8" -u "{M3u8_link}" -o "{output_path}"
for line in result:
    print(line)

# Record successful download to downloaded.txt
if os.path.exists(output_path):
    log_file = os.path.join(output_dir, "downloaded.txt")
    with open(log_file, "a", encoding="utf-8") as f:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"[{timestamp}] {filename} | {M3u8_link}\n")
    print(f"Downloaded to: {output_path}")
    print(f"Logged to: {log_file}")
else:
    print(f"Download failed: {output_path}")

In [ ]:
#@title ← Run <center> <font color='lightBlue'> Batch Download </font> </center> { display-mode: "form" }
#@markdown Enter filename and URL pairs in plain text format using `\n` for line breaks:
#@markdown ```
#@markdown "video1.mp4\nhttps://example.com/stream1.m3u8\nvideo2.mp4\nhttps://example.com/stream2.m3u8"
#@markdown ```
import os
from datetime import datetime

batch_input = "" #@param {type:"raw"}

lines = [l.strip() for l in batch_input.strip().split('\n') if l.strip()]
total = len(lines) // 2
success = 0
failed = []
log_file = os.path.join(output_dir, "downloaded.txt")

for i in range(0, len(lines), 2):
    if i + 1 < len(lines):
        fname = lines[i]
        url = lines[i + 1]
        output_path = os.path.join(output_dir, fname)
        print(f"[{i//2 + 1}/{total}] Downloading: {fname}")
        try:
            !"/content/m3u8" -u "{url}" -o "{output_path}"
            # Record successful download
            if os.path.exists(output_path):
                with open(log_file, "a", encoding="utf-8") as f:
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    f.write(f"[{timestamp}] {fname} | {url}\n")
                success += 1
            else:
                failed.append(fname)
        except Exception as e:
            print(f"Failed: {fname} - {e}")
            failed.append(fname)

print(f"\nCompleted: {success}/{total}")
if failed:
    print(f"Failed downloads: {', '.join(failed)}")
if success > 0:
    print(f"Download log: {log_file}")

In [ ]:
#@title ← Run <center> <font color='green'> Single File Download (yt-dlp) </font> </center> { display-mode: "form" }
import os
from datetime import datetime

M3u8_link_ytdlp = "" #@param {type:"string"}
filename_ytdlp = "video.mp4" #@param {type:"string"}

output_path = os.path.join(output_dir, filename_ytdlp)

!yt-dlp \
    --external-downloader aria2c \
    --external-downloader-args "-x 16 -s 16 -k 1M" \
    --merge-output-format mp4 \
    --progress \
    -o "{output_path}" \
    "{M3u8_link_ytdlp}"

# Record successful download to downloaded.txt
if os.path.exists(output_path):
    log_file = os.path.join(output_dir, "downloaded.txt")
    with open(log_file, "a", encoding="utf-8") as f:
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        f.write(f"[{timestamp}] {filename_ytdlp} | {M3u8_link_ytdlp}\n")
    print(f"Downloaded to: {output_path}")
    print(f"Logged to: {log_file}")
else:
    print(f"Download failed: {output_path}")

In [ ]:
#@title ← Run <center> <font color='orange'> Batch Download (yt-dlp) </font> </center> { display-mode: "form" }
#@markdown Enter filename and URL pairs in plain text format using `\n` for line breaks:
#@markdown ```
#@markdown "video1.mp4\nhttps://example.com/stream1.m3u8\nvideo2.mp4\nhttps://example.com/stream2.m3u8"
#@markdown ```
import os
from datetime import datetime

batch_input_ytdlp = "" #@param {type:"raw"}

lines = [l.strip() for l in batch_input_ytdlp.strip().split('\n') if l.strip()]
total = len(lines) // 2
success = 0
failed = []
log_file = os.path.join(output_dir, "downloaded.txt")

for i in range(0, len(lines), 2):
    if i + 1 < len(lines):
        fname = lines[i]
        url = lines[i + 1]
        output_path = os.path.join(output_dir, fname)
        print(f"[{i//2 + 1}/{total}] Downloading: {fname}")
        try:
            !yt-dlp \
                --external-downloader aria2c \
                --external-downloader-args "-x 16 -s 16 -k 1M" \
                --merge-output-format mp4 \
                --progress \
                -o "{output_path}" \
                "{url}"
            # Record successful download
            if os.path.exists(output_path):
                with open(log_file, "a", encoding="utf-8") as f:
                    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    f.write(f"[{timestamp}] {fname} | {url}\n")
                success += 1
            else:
                failed.append(fname)
        except Exception as e:
            print(f"Failed: {fname} - {e}")
            failed.append(fname)

print(f"\nCompleted: {success}/{total}")
if failed:
    print(f"Failed downloads: {', '.join(failed)}")
if success > 0:
    print(f"Download log: {log_file}")